### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [5]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [7]:
data.drop(['date','symbol'],axis=1).head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [56]:
data_feature = data[['open','low','high','volume']].head(1000)

In [57]:
data_target = data['close'].head(1000)

### Divide the data into train and test sets

In [58]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(data_feature, data_target, test_size=0.30, random_state=42)

## Building the graph in tensorflow

1.Define input data placeholders

In [59]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.nn.l2_normalize(x,1)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [60]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

3.Prediction

In [61]:
#We will use normalized data
#y = tf.add(tf.matmul(x,W),b,name='output')
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [62]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [63]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [64]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [65]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  7384.434
Training loss at step:  10  is  3402.8137
Training loss at step:  20  is  3093.9893
Training loss at step:  30  is  3070.0347
Training loss at step:  40  is  3068.1777
Training loss at step:  50  is  3068.032
Training loss at step:  60  is  3068.0227
Training loss at step:  70  is  3068.022
Training loss at step:  80  is  3068.025
Training loss at step:  90  is  3068.0205


In [66]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, test_loss = sess.run([train_op,loss],feed_dict={x:X_test, y_:y_test})
    
    if epoch % 10 == 0:
        print ('Testing loss at step: ', epoch, ' is ', test_loss)

Testing loss at step:  0  is  4186.908
Testing loss at step:  10  is  4175.0225
Testing loss at step:  20  is  4174.102
Testing loss at step:  30  is  4174.033
Testing loss at step:  40  is  4174.024
Testing loss at step:  50  is  4174.026
Testing loss at step:  60  is  4174.023
Testing loss at step:  70  is  4174.0225
Testing loss at step:  80  is  4174.0254
Testing loss at step:  90  is  4174.0254


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [21]:
sess.run(W) # value of W

array([[2.5700361e-03],
       [2.5463824e-03],
       [2.5909443e-03],
       [3.3388004e+01]], dtype=float32)

In [23]:
print(W) # shape of W

<tf.Variable 'Weights:0' shape=(4, 1) dtype=float32_ref>


In [22]:
sess.run(b) # value of b

array([33.388004], dtype=float32)

In [24]:
print(b) # shape of b

<tf.Variable 'Bias:0' shape=(1,) dtype=float32_ref>


### Find the Absolute mean square loss difference between training and testing loss.

In [48]:
train_loss

3068.021

In [49]:
test_loss

4174.0254

In [51]:
diff = abs(train_loss - test_loss)
diff

1106.0044

In [67]:
sess.close()

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [71]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1,activation='softmax'))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [84]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
700/700 [==============================] - 0s 49us/sample - loss: 7254.0377
Epoch 2/5
700/700 [==============================] - 0s 41us/sample - loss: 7254.0378
Epoch 3/5
700/700 [==============================] - 0s 39us/sample - loss: 7254.0379
Epoch 4/5
700/700 [==============================] - 0s 50us/sample - loss: 7254.0376
Epoch 5/5
700/700 [==============================] - 0s 49us/sample - loss: 7254.0377


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [76]:
iris = pd.read_csv('iris.csv')

### Splitting the data into feature set and target set

In [78]:
iris.drop('Id',axis=1).head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [85]:
iris_feature = iris[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]

In [80]:
iris_target = iris['Species']

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [83]:
iris_dum_target = pd.get_dummies(data=iris_target, columns=['Species'])

### Divide the dataset into Training and test (70:30)

In [86]:
X_train , X_test , y_train , y_test = train_test_split(iris_feature, iris_dum_target, test_size=0.30, random_state=42)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [119]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

model.add(tf.keras.layers.Dense(10))

model.add(tf.keras.layers.Dense(8))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(3,activation='softmax'))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])

### Fitting the model and predicting 

In [122]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
105/105 [==============================] - 0s 105us/sample - loss: 0.3336 - acc: 0.9048
Epoch 2/100
105/105 [==============================] - 0s 95us/sample - loss: 0.3234 - acc: 0.9143
Epoch 3/100
105/105 [==============================] - 0s 67us/sample - loss: 0.3224 - acc: 0.8667
Epoch 4/100
105/105 [==============================] - 0s 76us/sample - loss: 0.3476 - acc: 0.8952
Epoch 5/100
105/105 [==============================] - 0s 105us/sample - loss: 0.3713 - acc: 0.8476
Epoch 6/100
105/105 [==============================] - 0s 76us/sample - loss: 0.3044 - acc: 0.8857
Epoch 7/100
105/105 [==============================] - 0s 124us/sample - loss: 0.3111 - acc: 0.8857
Epoch 8/100
105/105 [==============================] - 0s 76us/sample - loss: 0.2912 - acc: 0.9333
Epoch 9/100
105/105 [==============================] - 0s 105us/sample - loss: 0.3134 - acc: 0.8667
Epoch 10/100
105/105 [==============================] - 0s 124us/sample - loss: 0.3133 - acc: 0.9143
Epoc

In [123]:
model.evaluate(X_test, y_test)

45/45 [==============================] - 0s 111us/sample - loss: 0.1444 - acc: 0.9556


[0.1444320301214854, 0.95555556]

In [124]:
y_pred = model.predict(X_test)

In [125]:
import numpy as np
y_p = np.argmax(y_pred,axis=1)
y_p

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 2, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 1, 0,
       0], dtype=int64)

In [126]:
import numpy as np
y_test.values
y_t = np.argmax(y_test.values,axis=1)
y_t

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0], dtype=int64)

### Report Accuracy of the predicted values

In [127]:
scores = model.evaluate(X_test, y_test)
print("Accuracy: ", scores[1])

45/45 [==============================] - 0s 89us/sample - loss: 0.1444 - acc: 0.9556
Accuracy:  0.95555556
